# EDGAR - Reading Information Tables in Text Format - Advanced Text Mining (25 marks)

#### So far, we have collected CIKs for each of the Mutual Funds, then looked up the links of all the 13F HRs and the Information Tables, and identified them either text tables or xml tables. In class, we  obtained the information from the xml tables.  In this assignment we will obtain information from the text file.  These are not as nicely structured as the xml output.   In a csv (HW_Mutual_Fund_Info_Table_Link.csv) you will find a few of these links.  Your goal for this part of the homework is to obtain the link to the text files from the attached file  (HW_Mutual_Fund_Info_Table_Link.csv).  Then you will write a code that will go to the links of all the linked text files, and extract some columns.  Do not use Beutiful Soup for this assignment.   We provide some initial code to guide your initial steps:

#### First we have to collect the links of the text Info Tables in lists:

In [1]:
import urllib.request

text_link = []
text_Name = []
text_date = []

input_file = open('/home/jovyan/OneDrive/DESKTOP/HW_Mutual_Fund_Info_Table_Link.csv', 'r')

rows = input_file.readlines()
input_file.close()

for item in range(1,len(rows)):
    columns = rows[item].split(',')
    if columns[4] == 'text':
        text_link.append(columns[3])
        text_Name.append(columns[1])
        text_date.append(columns[2])

# Your Code on Enumerating the Lists.  The result should be three lists, text_link, text_Name,
#and text_date.  Each should have length 122.

In [2]:
len(text_link)

122

In [3]:
len(text_Name)

122

In [4]:
len(text_date)

122

#### Keep only the links that correspond to a date after 2010 (Don't inlude 2010, start at 2011).  Hint: you can use the datetime library.

In [8]:
from datetime import datetime

#Use the following list to store the filtered values.
filtered_dates = []
filtered_name = []
filtered_link = []

#Enter code here to to keep only the dates corresponding to after 2010.  

start_date = datetime.strptime('2010-12-31', '%Y-%m-%d')
for item in rows[1:]:
    columns = item.split(',')
    year = datetime.strptime(columns[2], '%Y-%m-%d').year
    if columns[4] == 'text' and year > 2010:
        filtered_link.append('https://www.sec.gov' + columns[3])
        filtered_name.append(columns[1])
        filtered_dates.append(columns[2])


Your filtered list should now have 8 elements. These represent 3 mutual funds. The first one has CIK=1311981 (from the link you can find this at data/1311981 ). The second was has CIK 813470. The third one has CIK 1432353.
['https://www.sec.gov/Archives/edgar/data/1311981/000116204413000513/0001162044-13-000513.txt ', 'https://www.sec.gov/Archives/edgar/data/813470/000081347013000006/0000813470-13-000006.txt ', 'https://www.sec.gov/Archives/edgar/data/813470/000081347013000001/0000813470-13-000001.txt ', 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000023/0000813470-12-000023.txt ', 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000019/0000813470-12-000019.txt ', 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000014/0000813470-12-000014.txt ', 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000003/0000813470-12-000003.txt ', 'https://www.sec.gov/Archives/edgar/data/1432353/000114420411008428/0001144204-11-008428.txt ']

#### Next, for each text link, extract the name of issuer, CUSIP, and the Quantity of shares.  You will also want to keep track of the mutual fund name as well as the filing report date.  

#### Your output file should have 5 columns.  The first is the issue date of the form which can be found in the filtered_date list (this will be repeated for the same form).  The second is the mutual fund name which can be found in the filtered_name list (this will be repeated).  The third, fourth and fifith are the name of issuer, CUSIP, and shares respectively.  Make sure to account for the fact that while some of the text files have the same formatting, others do not.  This means you will have to look through them to make sure your code works for the each text file.

#### Finally, you write down the lists in the following file:

In [9]:
import urllib.request
import re

issue_date = []
mutual_fund_name = [] 
name_of_issuer = [] 
CUSIP = [] # CUSIP number
shares = [] # No. of Shares of the company in the Mutual Fund
for index in range(len(filtered_link)):
    url = filtered_link[index]
    txt = urllib.request.urlopen(url)
    txtLines = txt.readlines()
    table_flag = False
    columns_flag = False
    start_index = []
    for i in range(len(txtLines)):
        line = txtLines[i].decode("utf-8") 
        if line.find("<CAPTION>") > -1:
            table_flag = True
            continue
        if line.find("</TABLE>") > -1:
            table_flag = False
            columns_flag = False
            continue
        if table_flag:
            if columns_flag:
                if line[0] != ' ' and line[0] != '-' and line.find('NAME OF ISSUER') < 0:
                    if len(line.split()) < 2:
                        continue
                    issue_date.append(filtered_dates[index])
                    mutual_fund_name.append(filtered_name[index])
                    name_of_issuer.append(line[:start_index[1]].strip())
                    CUSIP.append(line[start_index[2]: start_index[3]])
                    shares.append(line[start_index[4]: start_index[5]])
            if line.find("<C>") > -1:
                start_index = []
                parttern = "<\w>"
                columns = re.finditer(parttern, line)
                for match in columns:
                    start_index.append(match.start())
                columns_flag = True
                continue
result = []
finalHeader = ['issue date', 'mutual fund name', 'name of issuer', 'CUSIP', 'shares']
headerLine = '\t'.join(finalHeader)
result.append(headerLine)
for i in range(len(issue_date)):
    entryLine = '\t'.join([
                str(issue_date[i]),
                str(mutual_fund_name[i]),
                str(name_of_issuer[i]),
                str(CUSIP[i]),
                str(shares[i])
            ])
    result.append(entryLine)
with open('./output.txt', 'a+') as f:
    for item in result:
        f.write("%s\n" % item)
    f.close()